In [1]:
import tensorflow as tf

In [2]:
import nltk

In [3]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from nltk.stem import PorterStemmer

In [5]:
stemmer = PorterStemmer()

In [6]:
import tensorflow as tf
import numpy as np
import random
import json

In [45]:
#from google.colab import files
#files.upload()
#with open("intents.json") as json_data:
#  intents = json.load(json_data)

In [46]:
intents={'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot','This bot was build in Python.',
    'I am a chatbot.'],
   'tag': 'chatbot'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from WWW.",
    'Visit compquant.com for meet me',
    'Thans for your Interest. I live in WWW.'],
   'tag': 'location'},
  {'patterns': ['Give me your social media accounts link',
    'Where can we connect',
    'How can i reach out to you?',
    'Is there any way we can connect'],
   'responses': ['You can connect me on Linkedin or Github',
    'You can visit https://compquant.com'],
   'tag': 'connect'},
  {'patterns': ['Which is your favourite movie?',
    'Suggest me some movies',
    'Recommend movies'],
   'responses': ['3 idiots',
    'Fast and Furious'],
   'tag': 'movies'},
  {'patterns': ['Who are you?', 'Tell me about Yourself', 'What is this?'],
   'responses': ['Hi, I am a bot. Nice to meet you. I made this chatbot for fun and practice.',
    'Thanks for asking. I am a Bot, for practice and learn tensorflow.'],
   'tag': 'about'}]}

##pre processing

In [47]:
words = []
classes = []
documents=[]
ignore=["?"]


In [48]:
for intent in intents["intents"]:
  print(intent)
  for pattern in intent["patterns"]:
    # tokenize each word in sentence
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w,intent["tag"]))

    if(intent["tag"] not in classes):
      classes.append(intent["tag"])

{'context_set': '', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'tag': 'greeting'}
{'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.'], 'tag': 'goodbye'}
{'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'tag': 'thanks'}
{'patterns': ['Who built this chatbot?', 'Tell me about Chatbot', 'What is this chatbot name?'], 'responses': ['Hi, I am Chatbot', 'This bot was build in Python.', 'I am a chatbot.'], 'tag': 'chatbot'}
{'patterns': ['What is your location?', 'Where are you located?', 'What is your address?'], 'responses': ['We are from WWW.', 'Visit compquant.com for meet me', 'Thans for your Interest. I live in WWW.'], 'tag': 'location'}
{'patterns': ['Give me your social media accounts link',

In [49]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents),"documents",documents)
print(len(classes), "classes",classes)
print(len(words), "unique stemmed words",words)

27 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Who', 'built', 'this', 'chatbot', '?'], 'chatbot'), (['Tell', 'me', 'about', 'Chatbot'], 'chatbot'), (['What', 'is', 'this', 'chatbot', 'name', '?'], 'chatbot'), (['What', 'is', 'your', 'location', '?'], 'location'), (['Where', 'are', 'you', 'located', '?'], 'location'), (['What', 'is', 'your', 'address', '?'], 'location'), (['Give', 'me', 'your', 'social', 'media', 'accounts', 'link'], 'connect'), (['Where', 'can', 'we', 'connect'], 'connect'), (['How', 'can', 'i', 'reach', 'out', 'to', 'you', '?'], 'connect'), (['Is', 'there', 'any', 'way', 'we', 'can', 'connect'], 'connect'), (['Which', 'is', 'your', 'favourite', 'movie', '?'], 'mov

In [50]:
# creating traning model

In [51]:
training = []
output = []
output_empty =[0]*len(classes)


In [52]:
# create training set, bag of words

In [53]:
for doc in documents:
  bag = []
  # list of tokenized words for the pattern
  pattern_words =doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  #bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag,output_row])
  

In [54]:
random.shuffle(training)

In [55]:
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [56]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [57]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]),activation="softmax"))
model.compile(tf.keras.optimizers.Adam(),loss="categorical_crossentropy",metrics=["accuracy"])


In [21]:
model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size=8,verbose=1)
model.save("model.pkl")

Epoch 1/100
4/4 [==============================] - 1s 4ms/step - loss: 2.1696 - accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1387 - accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 6ms/step - loss: 2.1120 - accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0883 - accuracy: 0.0370
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0658 - accuracy: 0.0370
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0440 - accuracy: 0.0370
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0218 - accuracy: 0.0370
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0006 - accuracy: 0.0741
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9797 - accuracy: 0.0741
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9584 - accuracy: 0.0741
Epoch 11/100
4/4 

In [22]:
import pickle

In [23]:
pickle.dump({"words":words,"classes":classes},open("training_data","wb"))

In [24]:
from keras.models import load_model

In [25]:
model = load_model("model.pkl")

In [26]:
data = pickle.load(open("training_data","rb"))
words =data["words"]
classes = data["classes"]


In [27]:
#with open("intents.json") as json_data:
#  intents = json.load(json_data)

In [28]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

In [29]:
def bow(sentence,words):
  sentence_words=clean_up_sentence(sentence)
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
  bag = np.array(bag)
  return bag


In [33]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
  bag = bow(sentence,words)
  results = model.predict(np.array([bag]))
  results = [ [i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD  ]
  results.sort(key=lambda x:x[1],reverse=True)
  return_list =[]
  for r in results:
    return_list.append((classes[r[0]],r[1]))
  return return_list

In [42]:
def response(sentence):
  results = classify(sentence)
  if results:
    while results:
      for i in intents["intents"]:
        if(i["tag"]==results[0][0]):
          
          return print(random.choice(i["responses"]))
      results.pop(0)

In [43]:
response("Where are you located?")

We are from World's largest Democracy India.


In [44]:
response("Bye!")

Have a nice day


In [ ]:
# try different architecture
# try more data
# try different NLP technique